# d7046e ANN course project , Group 6

The project goal is to implement a classifier for clear/cloudy sky based satellite image data. 

**Download and some checks**

In [ ]:
# downloads and extract the data from, https://drive.google.com/drive/folders/1lRCIcQo9CqFRDhUd3aZRAA46k8nLL49J

import gdown
import zipfile
import os

CLOUD_FILE_ID = "19MBh9JIJTxYIPAeO7G5RML5_ddjJ1Cpa"
CLOUD_DOWN_ENDPOINT = "https://drive.google.com/uc?id="

base_dir = os.path.abspath("../data/")  
zip_path = os.path.join(base_dir, "data.zip")  
extract_path = os.path.join(base_dir) # set to same not to create to many sub folders 

os.makedirs(extract_path, exist_ok=True)

print(f"downloading dataset to: {zip_path}")
gdown.download(f"{CLOUD_DOWN_ENDPOINT}{CLOUD_FILE_ID}", zip_path, quiet=False)

print(f"extracting to: {extract_path}")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

os.remove(zip_path)
print(f"deleted zip file: {zip_path}")

print("download and extraction completed.")
print(f"extracted files in: {extract_path}")
print(f"extracted content: {os.listdir(extract_path)}")


downloading dataset to: c:\Users\moell\dev\dev_uni\d7046e\project\d7046e_spaceDataAnalysis_project\data\data.zip


Downloading...
From: https://drive.google.com/uc?id=19MBh9JIJTxYIPAeO7G5RML5_ddjJ1Cpa
To: c:\Users\moell\dev\dev_uni\d7046e\project\d7046e_spaceDataAnalysis_project\data\data.zip
100%|██████████| 4.79M/4.79M [00:00<00:00, 10.0MB/s]


extracting to: c:\Users\moell\dev\dev_uni\d7046e\project\d7046e_spaceDataAnalysis_project\data
deleted zip file: c:\Users\moell\dev\dev_uni\d7046e\project\d7046e_spaceDataAnalysis_project\data\data.zip
download and extraction completed.
extracted files in: c:\Users\moell\dev\dev_uni\d7046e\project\d7046e_spaceDataAnalysis_project\data
extracted content: ['README.md', 'skogsstyrelsen-data']


<small>Trying to figure out how the data is structured.</small>

In [10]:
import numpy as np

# length of gts files, trying to find out what to use and make sure where tha labels are.
train_gts = np.load(os.path.join(base_dir, "skogsstyrelsen-data/skogs_gts_train.npy"))
train_names = np.load(os.path.join(base_dir, "skogsstyrelsen-data/skogs_names_train.npy"))
print(f"length of train_gts:{len(train_gts)} , length of train_names:{len(train_names)}")

val_gts = np.load("../data/skogsstyrelsen-data/skogs_gts_val.npy")
val_names = np.load("../data/skogsstyrelsen-data/skogs_names_val.npy")
print(f"length of train_gts:{len(val_gts)} , length of train_names:{len(val_names)}")


length of train_gts:260 , length of train_names:260
length of train_gts:72 , length of train_names:72


<small>Sample one of the objects in the .nc files</small>

In [4]:
# print a .nc array 
import xarray as xar
ds = xar.open_dataset("../data/skogsstyrelsen-data/2A-netcdfs-cropped-from-nuria/skgs_0b5101fb-44c7-ed11-9174-005056a6f472.nc") # first file from the download folder
print(ds)

scl_value = ds['scl'].values
print(f"\nscl_value:\n{scl_value}")

bands = []

# print some of the object attributes, familiarize with the structure of the data
for band in ds.data_vars:
    data_array = ds[band]
    values = data_array.values
    bands.append(values)

    print(f"Variable name: {band}")
    print(f"Shape: {values.shape}")
    print(f"Dtype: {values.dtype}\n")

print(f"bands array:\n{bands}")


<xarray.Dataset> Size: 22kB
Dimensions:      (time: 1, y: 21, x: 20)
Coordinates:
  * time         (time) datetime64[ns] 8B 2020-08-11T10:05:59
  * y            (y) float64 168B 6.558e+06 6.558e+06 ... 6.558e+06 6.558e+06
  * x            (x) float64 160B 5.618e+05 5.618e+05 ... 5.616e+05 5.616e+05
    spatial_ref  int32 4B ...
Data variables: (12/13)
    b01          (time, y, x) float32 2kB ...
    b02          (time, y, x) float32 2kB ...
    b03          (time, y, x) float32 2kB ...
    b04          (time, y, x) float32 2kB ...
    b05          (time, y, x) float32 2kB ...
    b06          (time, y, x) float32 2kB ...
    ...           ...
    b08          (time, y, x) float32 2kB ...
    b8a          (time, y, x) float32 2kB ...
    b09          (time, y, x) float32 2kB ...
    scl          (time, y, x) float32 2kB ...
    b11          (time, y, x) float32 2kB ...
    b12          (time, y, x) float32 2kB ...
Attributes:
    date_created:           2023-06-28T13:14:56.156801
    C

**Setup Cells**

In [ ]:
import torch
import xarray as xar
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

DATA_PATH = "../data/skogsstyrelsen-data/"
BEST_MODEL_BASE_PATH = "stored/"

BANDS = ['b01', 'b02', 'b03', 'b04', 'b05', 'b06', 'b07', 'b08', 'b8a', 'b09', 'b11', 'b12']
IMAGE_SIZE = 20 # bands array from the above print seems to have shape 20X20, 2D 
EPOCHS = 50
BATCH_SIZE = 32
LR = 1e-3


if torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"
    print(
        "Cuda does not seem to be available in your environment, try installing a torch package compiled with cuda enabled if u have cuda installed in ur os.\n"
        "Uninstall torch, torchaudio, torchvision, and then run: torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121\n"
    )
    
print(f"Running with: DEVICE = {DEVICE}")

Running with: DEVICE = cuda


<small>Utils functions to load the data from skogs</small>

In [ ]:

def get_skogs_paths(split_suffix: str):
    """ Returns concatenated paths depending on what split that is needed. """
    names_path = os.path.join(DATA_PATH, f"skogs_names_{split_suffix}.npy")
    labels_path = os.path.join(os.path.join(DATA_PATH, f"skogs_gts_{split_suffix}.npy"))
    return names_path, labels_path

def load_skogs_data(split_suffix: str):
    """Loads a dataset split into numpy arrays for images and binary labels."""
    names_path, labels_path = get_skogs_paths(split_suffix)
    names = np.load(names_path, allow_pickle=True)                       # list of paths to nc files
    labels = np.load(labels_path, allow_pickle=True).astype(np.float32)  # binary values from the gts files

    x_list = []
    y_list = []

    for name, label in zip(names, labels):  
        nc_file = os.path.join(DATA_PATH, "2A-netcdfs-cropped-from-nuria", os.path.basename(name)) # could not get the path to work with just the path from names file
        nc_file = os.path.normpath(nc_file)  # normalize path for different OS

        if os.path.exists(nc_file):
            with xar.open_dataset(nc_file, engine="netcdf4") as ds:
                band_arrays = [(ds[band].values.squeeze() - 1000) / 10000 for band in BANDS]

            # reshape bands of different shapes 
            band_arrays_fixed = []
            for band in band_arrays:
                if band.shape[0] >= IMAGE_SIZE and band.shape[1] >= IMAGE_SIZE:
                    band_fixed = torch.tensor(band[:IMAGE_SIZE, :IMAGE_SIZE], dtype=torch.float32) # slice the top left part of the image consistently, 
                else:                                                                              # mby need to change the slice if important features are lost
                    band_fixed = torch.zeros((IMAGE_SIZE, IMAGE_SIZE), dtype=torch.float32)
                
                band_arrays_fixed.append(band_fixed)

                x = torch.stack(band_arrays_fixed)  # should be of size (batch_size, 20, 20) when stacking
        else:
            print(f"File not found: {nc_file}")
            x = torch.zeros((len(BANDS), IMAGE_SIZE, IMAGE_SIZE), dtype=torch.float32) 

        x_list.append(x.numpy())  
        y_list.append(label)  

    return np.array(x_list), np.array(y_list)  

class SkogsBinaryDataset(Dataset):
    """ PyTorch dataset for the stacked band arrays (X) and binary label (y). """
    def __init__(self, x_data, y_data):
        self.x_data = x_data # tensor for the image
        self.y_data = y_data # label for the image

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = torch.tensor(self.x_data[idx], dtype=torch.float32)  # shape: (bands, rows, cols), ((72), 12, 21, 20)
        y = torch.tensor(self.y_data[idx], dtype=torch.float32)  # shape: (72), just the labels from the gts file, 72 images in val set
        return x, y


<small>Set up data loaders, this cell is just a dev test to see that the data extraction is working</small>

In [ ]:
def check_batch_shapes(ds: DataLoader):
    for x_batch, y_batch in ds:
        print(f"x_batch shape: {x_batch.shape}") # (batch_size, channels, height, width)
        print(f"y_batch shape: {y_batch.shape}") # (batch_size) , same amount of labels as input images

x_train, y_train = load_skogs_data("train")
x_val, y_val = load_skogs_data("val")
x_test, y_test = load_skogs_data("test")

train_ds = SkogsBinaryDataset(x_train, y_train)
val_ds = SkogsBinaryDataset(x_val, y_val)
test_ds = SkogsBinaryDataset(x_test, y_test)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True) 
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

# uncomment if u want to check shapes
#check_batch_shapes(train_loader)
#check_batch_shapes(val_loader)
#check_batch_shapes(test_loader)


x_batch shape: torch.Size([32, 12, 20, 20])
y_batch shape: torch.Size([32])
x_batch shape: torch.Size([32, 12, 20, 20])
y_batch shape: torch.Size([32])
x_batch shape: torch.Size([32, 12, 20, 20])
y_batch shape: torch.Size([32])
x_batch shape: torch.Size([32, 12, 20, 20])
y_batch shape: torch.Size([32])
x_batch shape: torch.Size([32, 12, 20, 20])
y_batch shape: torch.Size([32])
x_batch shape: torch.Size([32, 12, 20, 20])
y_batch shape: torch.Size([32])
x_batch shape: torch.Size([32, 12, 20, 20])
y_batch shape: torch.Size([32])
x_batch shape: torch.Size([32, 12, 20, 20])
y_batch shape: torch.Size([32])
x_batch shape: torch.Size([4, 12, 20, 20])
y_batch shape: torch.Size([4])


**Model architecture 1**

In [ ]:
class BinCloudCnn(nn.Module):
    def __init__(self, hidden_activation=F.relu, output_activation=torch.sigmoid, stride=1):  # the standard values for the activations are set as a dev reference, call with different values and eval
        """ call the class with different   """
        super(BinCloudCnn, self).__init__()

        self.activation_fn = hidden_activation
        self.output_activation = output_activation

        self.conv1 = nn.Conv2d(in_channels=12, out_channels=32, kernel_size=3, padding=1)  # (32, 20, 20)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)  # (64, 10, 10)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=stride)

        self.fc1 = nn.Linear(64 * 10 * 10, 128)
        self.fc2 = nn.Linear(128, 1)  # fully con output layer

    def forward(self, x):
        x = self.conv1(x)
        x = self.activation_fn(x)  
        x = self.pool(x)

        x = self.conv2(x)
        x = self.activation_fn(x)  
        x = self.pool(x)

        x = x.view(x.size(0), -1)

        # maybe add dropouts here
        x = self.fc1(x)
        x = self.activation_fn(x)  
        x = self.fc2(x)
        
        x = self.output_activation(x) 

        return x

**Training an Validation functions**

In [ ]:
import time
import torch.optim.optimizer

def validate(
    model: BinCloudCnn,
    val_loader: DataLoader,
    loss_criterion: torch.nn.Module,  # any loss criterion from torch.nn
    device: str = DEVICE
):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad(): 
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = loss_criterion(outputs, targets)
            val_loss += loss.item()

            predicted = (outputs > 0.5).float()
            correct += (predicted == targets.unsqueeze(1)).sum().item()
            total += targets.size(0)

    avg_val_loss = val_loss / len(val_loader)
    accuracy = correct / total * 100

    return avg_val_loss, accuracy


def train(
        model: BinCloudCnn,
        train_loader: DataLoader,
        val_loader: DataLoader,
        loss_criterion: torch.nn.Module,    # any loss criterion from torch.nn
        optimizer: torch.optim.Optimizer,   # any optimizer from torch.optim
        epochs: int = EPOCHS,
        patience: int = 10,
        device: str = DEVICE,
        best_model_path: str = "best_arch1_model.pth",     
    ):
    
    best_model_path = f"{BEST_MODEL_BASE_PATH}{best_model_path}"
    start_time = time.time()

    model.to(device)
    print(f"model loaded to device: {device}\n")

    best_val_loss = float("inf")
    best_model_state = None
    patience_count = 0

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    # epoch loop, wraps the validation function
    for epoch in range(epochs):
        loss: torch.Tensor = 0 # just declare type for ide support
        total_correct = 0
        total_samples = 0
        total_train_loss = 0

        model.train()
        for batch in train_loader:
            image: torch.Tensor
            labels: torch.Tensor
            image, labels = batch  # the tuple returned from SkogsBinaryDataset
            image = image.to(device)
            labels = labels.to(device)

            optimizer.zero_grad() # gradient reset
            outputs = model(image)

            loss = loss_criterion(outputs, labels.float().unsqueeze(1)) 
            loss.backward()
            optimizer.step() # update w

            total_train_loss +=  loss.item()

            # convert output to binary 
            predictions = (outputs >= 0.5).float()
            total_correct += (predictions == labels.unsqueeze(1)).sum().item()
            total_samples += labels.size(0)

        avg_train_loss = total_train_loss / len(train_loader)
        epoch_train_accuracy = total_correct / total_samples

        train_losses.append(avg_train_loss)
        train_accuracies.append(epoch_train_accuracy)

        # the validation call, sets the mode and stuff
        epoch_val_loss, epoch_val_accuracy = validate(model, val_loader, loss_criterion, device)
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_accuracy)

        # epoch report
        print(f"Epoch {epoch+1}/{epochs} | TrainLoss = {avg_train_loss:.4f}, TrainAccuracy = {epoch_train_accuracy} | ValLoss = {epoch_val_loss} , ValAccuracy = {epoch_val_accuracy}")


        # save the best model for the run
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            best_model_state = model.state_dict()
            patience_count = 0 # reset patience!!!
        else:
            patience_count += 1
    
        if patience_count >= patience:
            print(f"--- early stopping triggered at epoch {epoch + 1}, best ValLoss = {best_val_loss:.4f} ---")
            print_run_time(start_time, True)
            break

        if best_model_state is not None:
            torch.save(best_model_state, best_model_path)
            print(f"saved best model to path: {best_model_path}")

    print_run_time(start_time, False)
        


def print_run_time(start_time, is_early):
    end_time = time.time()
    train_val_time = end_time - start_time 
    if is_early:
        print(f"training + validation early stop time = {train_val_time:.2f} s")
        return 
    print(f"training + validation no early stopping time = {train_val_time:.2f} s") 



<small>The testing function</small>

In [ ]:
# put the testing here deven

**Train Architecture1**

In [ ]:

# setup the data loaders for training
x_train, y_train = load_skogs_data("train")
x_val, y_val = load_skogs_data("val")
x_test, y_test = load_skogs_data("test")

train_ds = SkogsBinaryDataset(x_train, y_train)
val_ds = SkogsBinaryDataset(x_val, y_val)
test_ds = SkogsBinaryDataset(x_test, y_test)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True) 
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)


first_model = BinCloudCnn(

)

optimizer = torch.optim.Adam(first_model.parameters(), lr=LR)
